# 滴滴比赛

导入必须的！

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandasql as pql
from datetime import datetime
%matplotlib inline

### 文件位置

In [2]:

# 训练集数据文件夹
base_dir = "G:/project/dataset/didi/season_1/training_data"

train_cluster_map_file = os.path.join(base_dir, 'cluster_map', 'cluster_map')

# 这个 POI 不打算使用
train_poi_file = os.path.join(base_dir, 'poi_data', 'poi_data')

# 训练集的 gap 中间文件
train_sd_gap_file = os.path.join(base_dir, 'sd_gap.csv')

# 训练集的 order 中间文件
train_order_file = os.path.join(base_dir, 'order_data', 'order_data.csv')

train_weather_file = os.path.join(base_dir, 'weather_data', 'weather_data.csv')

train_traffic_file = os.path.join(base_dir, 'traffic_data', 'traffic-data.csv')

# test data directory
test_base_dir = "G:/project/dataset/didi/season_1/test_set_2"

test_traffic_file = os.path.join(base_dir, 'traffic_data', 'traffic-data.csv')
test_order_file = os.path.join(test_base_dir, 'order_data', 'order-data.csv')

test_submit_file = os.path.join(test_base_dir, 'test.txt')

test_weather_file = os.path.join(test_base_dir, 'weather_data', 'weather_data.csv')

# 全部交通数据
all_traffic_file = os.path.join(base_dir, '..', 'traffic.csv')

sd_gap_file = os.path.join(base_dir, '..', 'gap.csv')

weather_file = os.path.join(test_base_dir, '..', 'weather_data.csv')

another_weather = os.path.join(test_base_dir, '..', 'weather.txt')
full_index_file = os.path.join(base_dir, '..', 'fullindex.csv')

full_file = os.path.join(base_dir, '..', 'full_data.csv')

# 提交文件
submit_file = os.path.join(test_base_dir, '..', 'submit.csv')
result_file = os.path.join(test_base_dir, '..', 'result.csv')


** 强调中间步骤、 中间文件 **

在最终的模型预测之前， 数据预处理、 分析时， 多使用中间文件保存结果

避免重复执行步骤

# 完整DataFrame

66个区域 * 21（21+5）天 * 144 个时间片， 再把各项数据并入

有完整的， 才知道缺了哪些值， 才能填充—— 我目前想到的办法， 个人觉得比 在原dataframe 上一行行地添加数据来得快。

In [10]:
def generate_full_df(day):
    
    time = range(1, 145)
    
    regid = np.array([])
    dates = np.array([])
    times = np.array([])

    for d in day:
        dates = np.hstack((dates, np.array(['2016-01-%02d' % d] * 66 * 144)))
        for i in range(66):
            regid = np.hstack((regid, np.array([i+1] * 144)))
            times = np.hstack((times, time))

    #print len(dates), len(regid), len(times)
    full_df = pd.DataFrame({'id': regid, 'Date': dates, 'TimePiece': times}, columns=['id', 'Date', 'TimePiece'])
    print len(full_df)
    full_index_file = os.path.join(base_dir, '..', 'fullindex2.csv')
    full_df.to_csv(full_index_file, index=False)
    return full_df

In [11]:
day = range(1, 23) + [23, 25, 27, 29, 30]
full_df = generate_full_df(day)

256608


## 从中间文件读取

读取中间文件 full_index_file

In [3]:
full_df = pd.read_csv(full_index_file, dtype = {'id': np.int16, 'TimePiece': np.int16})
print len(full_df)

247104


In [6]:
full_df[:3]

,id,Date,TimePiece
0,1,2016-01-01,1
1,1,2016-01-01,2
2,1,2016-01-01,3


# cluster_map 处理

## 区域 hashid 和 实际id

cluster_map 数据， pandas 应该比 dict 快， 到时候 merge 或 join 比 从 dict 里找好。

In [3]:
cluster_map = pd.read_csv(train_cluster_map_file, sep="\t", header=None, names=['hashid', 'id'])

In [9]:
cluster_map[:2]

,hashid,id
0,90c5a34f06ac86aee0fd70e2adce7d8a,1
1,f2c8c4bb99e6377d21de71275afd6cd2,2


# 订单 order_data 处理

## 复习下 pandas 读数据

1. 复习下数据读取, read_csv
2. 个人认为不需要的数据， 如 order_id 等id， 该省略的省掉（order_id, passenger_id, dest_district_has）， 该转换的转换, driver_id 变成是否有接单， 时间拆出日期和时间片， start_distrct_hash 改成实际id。 节省空间。去掉一堆 hash的字符串， 可以省很多空间

**也许有些信息还有用， 觉得有用时再补充回来**


In [4]:
# 时：分：秒 转成 10分钟的时间片
def time2Slice(time_str):
    l = time_str.split(":")
    return int(l[0]) * 6 + int(l[1]) / 10 + 1

In [5]:
# Order data 的数据 hash 太多， 顺便复习下 pandas
def reduceOrderFile(base_name, f):
    import time
    prev = time.time()
    fname = os.path.join(base_name, f)
    order = pd.read_csv(fname, sep="\t", header=None, dtype={'Price': np.str},
                          names=['order_id', 'driver_id', 'passenger_id', 'start_district_hash', 'dest_district_hash',
                                'Price', 'Time'])

    full_order = order.merge(cluster_map, left_on='start_district_hash', right_on='hashid')

    full_order['Date'] = pd.Series([e.split()[0] for e in full_order['Time']])

    full_order['TimePiece'] = pd.Series([time2Slice(e.split()[1]) for e in full_order['Time']])

    full_order['Resp'] = full_order['driver_id'].notnull()

    small_order = full_order.loc[:, ['Date', 'TimePiece', 'id', 'Price', 'Resp']]
    small_order_file = os.path.join(base_name, f + ".csv")
    small_order.to_csv(small_order_file, index = False)
    print time.time() - prev
    
    #return small_order

In [7]:
# 测试函数

def testReduceOrder():
    z = reduceOrderFile('G:/project/dataset/citydata/season_1/training_data\order_data', 'order_data_2016-01-21')
    print len(z)
    print sum(z['Resp'])

In [6]:
# 批量 order reduce
def reduceFiles(base_name):
    for rt, dirs, files in os.walk(base_name):
        for f in files:           
            reduceOrderFile(base_name, f)


In [7]:
# 运行过就不要再来了， 一个文件6秒， 20个文件， 还是要2分钟的。
def runForOnce():
    pass
reduceFiles(os.path.join(test_base_dir, 'order_data'))

0.074000120163
0.0299999713898
0.000999927520752
0.0
0.0160000324249
1.20499992371
1.21799993515
1.25100016594
1.41599988937
1.11100006104


In [43]:
def test_read_order():
    train_order = pd.read_csv(train_order_file, sep="\t", header=None, dtype={'Price': np.float, 'driver_id': np.str},
                          names=['order_id', 'driver_id', 'passenger_id', 'start_district_hash', 'dest_district_hash',
                                'Price', 'Time'])
    print len(train_order)
    t = train_order['driver_id']
    print t.notnull()
    small_order = full_train_order.loc[:, ['Date', 'TimePiece', 'id', 'Price', 'Resp']]

# 读取 order 数据

训练和测试数据

训练数据 训练 验证

In [6]:
def order2SDGap(filename):
    order = pd.read_csv(filename, dtype={'TimePiece': np.int16, 'id': np.int16, 'Price': np.float, 'Resp': np.bool})
    
    group_all = order.loc[:, ['id', 'Date', 'TimePiece', 'Resp']].groupby(['id', 'Date', 'TimePiece'])
    sd_data = group_all.count()
    sd_data['supply'] = group_all.Resp.sum()
    sd_data['gap'] = sd_data.Resp - sd_data.supply
    sd_data.columns = ['demand', 'supply', 'gap']
    
    sd_df = sd_data.reset_index(level = ['id', 'Date', 'TimePiece'])
    sd_df['week'] = pd.Series([datetime.strptime(e, "%Y-%m-%d").weekday() for e in sd_df.Date])

    return sd_df

In [7]:
train_order = order2SDGap(train_order_file)
test_order = order2SDGap(test_order_file)

In [ ]:
test_order.

## 生成订单的供需缺口中间文件

In [25]:
train_order.to_csv(train_sd_gap_file, index=False)

In [8]:
train_order[:5]

,id,Date,TimePiece,demand,supply,gap,week
0,1,2016-01-01,1,187,178,9,4
1,1,2016-01-01,2,198,191,7,4
2,1,2016-01-01,3,192,182,10,4
3,1,2016-01-01,4,172,167,5,4
4,1,2016-01-01,5,153,152,1,4



## 从订单供需缺口 中间文件恢复数据

对应文件 sd_gap_file,

训练集对应文件 train_sd_gap_file


In [3]:
all_sd_df = pd.read_csv(sd_gap_file, dtype = {'id': np.int16, 'TimePiece': np.int16, 'demand': np.int32, 
                                                'supply': np.int32, 'gap': np.int32, 'week': np.int8})

In [4]:
test_data = all_sd_df[all_sd_df.Date > '2016-01-21'].loc[:, all_sd_df.columns]


array(['2016-01-22', '2016-01-24', '2016-01-26', '2016-01-28', '2016-01-30'], dtype=object)

In [5]:
test_data.TimePiece.unique()

array([ 43,  44,  45,  55,  56,  57,  67,  68,  69,  79,  80,  81,  91,
        92,  93, 103, 104, 105, 115, 116, 117, 127, 128, 129, 139, 140, 141], dtype=int64)

## 1月3号以后数据比较好

In [7]:
data = all_sd_df[all_sd_df.Date > '2016-01-03']
data[:3]

,id,Date,TimePiece,demand,supply,gap,week
1008,1,2016-01-08,1,76,72,4,4
1009,1,2016-01-08,2,51,50,1,4
1010,1,2016-01-08,3,54,53,1,4


In [8]:
data.describe()

,id,TimePiece,demand,supply,gap,week
count,109059.000000,109059.000000,109059.000000,109059.000000,109059.000000,109059.000000
mean,31.771041,78.345666,54.242905,44.511402,9.731503,3.006969
std,18.573738,39.415784,114.400586,83.286968,46.109139,2.002055
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,16.000000,49.000000,4.000000,3.000000,0.000000,1.000000
50%,31.000000,80.000000,12.000000,10.000000,1.000000,3.000000
75%,47.000000,112.000000,48.000000,43.000000,4.000000,5.000000
max,66.000000,144.000000,1863.000000,902.000000,1341.000000,6.000000


# 交通情况处理

## 读取文件

In [17]:
train_traffic = pd.read_csv(train_traffic_file, sep="\t", header=None,
                          names=['hashid', 'traffic1', 'traffic2', 'traffic3', 'traffic4', 'Time'])

In [18]:
len(train_traffic)

193553

## 交通拥堵情况评估标准

1. 四个等级
2. t1,..,t4 各自路段数

$ value = (t1 * 1 + t2 * 2 + t3 * 3 + t4 * 4) * 0.1 / (t1 * t2 * t3 * t4) $ 

再用 max, min 归一化

In [113]:
def deal_traffic(traffic_file):
    traffic_data = pd.read_csv(traffic_file, sep="\t", header=None,
                          names=['hashid', 'traffic1', 'traffic2', 'traffic3', 'traffic4', 'Time'])
    traffic_data['Date'] = pd.Series([e.split()[0] for e in traffic_data['Time']])
    traffic_data['TimePiece'] = pd.Series([time2Slice(e.split()[1]) for e in traffic_data['Time']])
    traffic_data.traffic1 = pd.Series([ int(e.split(':')[1]) for e in traffic_data.traffic1])
    traffic_data.traffic2 = pd.Series([ int(e.split(':')[1]) for e in traffic_data.traffic2])
    traffic_data.traffic3 = pd.Series([ int(e.split(':')[1]) for e in traffic_data.traffic3])
    traffic_data.traffic4 = pd.Series([ int(e.split(':')[1]) for e in traffic_data.traffic4])
    
    traffic_data['traffic'] = pd.Series((traffic_data.traffic1 + traffic_data.traffic2 * 2 + traffic_data.traffic3 * 3 + traffic_data.traffic4 * 4)
                                  * 0.1 /(traffic_data.traffic1 + traffic_data.traffic2 + traffic_data.traffic3 + traffic_data.traffic4) )
    traffic_min = traffic_data['traffic'].min()
    traffic_max = traffic_data['traffic'].max()

    print traffic_max, traffic_min
    
    traffic_data['traffic'] = (traffic_data['traffic'] - traffic_min) / (traffic_max - traffic_min)
    
    small_traffic = traffic_data.loc[:, ['hashid', 'Date', 'TimePiece', 'traffic']]
    return small_traffic.sort_values(['hashid', 'Date', 'TimePiece'])

In [114]:
# all_traffic_file
all_traffic = deal_traffic(all_traffic_file)

0.4 0.1


In [116]:
all_traffic.describe()

,TimePiece,traffic
count,201934.000000,201934.000000
mean,73.537913,0.094944
std,41.009222,0.043510
min,2.000000,0.000000
25%,39.000000,0.066667
50%,74.000000,0.093333
75%,109.000000,0.122040
max,144.000000,1.000000


## 交通情况中间文件

对应文件 traffic_file

In [118]:
all_traffic.to_csv(traffic_file, index = False)

## 从交通情况中间文件恢复数据

In [13]:
all_traffic = pd.read_csv(all_traffic_file, dtype={'TimePiece': np.int16})
#all_traffic['week'] = pd.Series([datetime.strptime(e, "%Y-%m-%d").weekday() for e in all_traffic.Date])
#groupWeek = train_traffic.groupby('week')
all_traffic[:4]

,hashid,Date,TimePiece,traffic
0,08232402614a9b48895cc3d0aeb0e9f2,2016-01-01,2,0.075000
1,08232402614a9b48895cc3d0aeb0e9f2,2016-01-01,3,0.127854
2,08232402614a9b48895cc3d0aeb0e9f2,2016-01-01,4,0.161994
3,08232402614a9b48895cc3d0aeb0e9f2,2016-01-01,5,0.117460


In [14]:
all_traffic.describe()

,TimePiece,traffic
count,201934.000000,201934.000000
mean,73.537913,0.094944
std,41.009222,0.043510
min,2.000000,0.000000
25%,39.000000,0.066667
50%,74.000000,0.093333
75%,109.000000,0.122040
max,144.000000,1.000000


## 完整交通情况填充

若干日期 * 66个区域 * 144 个时间片（30-144必需）

In [15]:
all_traffic = all_traffic.merge(cluster_map, left_on='hashid', right_on='hashid').loc[:, ['id', 'Date', 'TimePiece', 'traffic']]
all_traffic[:3]

,id,Date,TimePiece,traffic
0,50,2016-01-01,2,0.075000
1,50,2016-01-01,3,0.127854
2,50,2016-01-01,4,0.161994


In [24]:
print len(all_traffic)

201934


# 天气数据

weather_data.csv 文件

another_weather

In [16]:
weather_df = pd.read_csv(another_weather, dtype={'TimePiece': np.int16, 'Weather': np.int8})
weather_df.describe()

,TimePiece,Weather
count,3744.000000,3744.000000
mean,72.500000,2.379274
std,41.573769,1.928555
min,1.000000,0.000000
25%,36.750000,1.000000
50%,72.500000,2.000000
75%,108.250000,4.000000
max,144.000000,9.000000


In [73]:
for w, g in weather_df.groupby('Date'):
    
    if len(g) == 144:
        continue
    print w, len(g)
    t = 0
    for ww, gg in g.groupby('TimePiece'):
        if ww - t != 1:
            print ww
        t = ww

2016-01-21 143
43


In [49]:
print 144*26
144*26 * 66

3744


247104

# 整合全体数据

In [17]:
temp_df = full_df.merge(weather_df, on=['Date', 'TimePiece'], how='left')
temp_df.describe()

,id,TimePiece,Weather
count,247104.00000,247104.000000,247104.000000
mean,33.50000,72.500000,2.379274
std,19.05041,41.568301,1.928301
min,1.00000,1.000000,0.000000
25%,17.00000,36.750000,1.000000
50%,33.50000,72.500000,2.000000
75%,50.00000,108.250000,4.000000
max,66.00000,144.000000,9.000000


In [18]:
temp_df[:5]

,id,Date,TimePiece,Weather
0,1,2016-01-01,1,1
1,1,2016-01-01,2,1
2,1,2016-01-01,3,1
3,1,2016-01-01,4,1
4,1,2016-01-01,5,1


In [65]:
for n, g in temp_df.groupby('Date'):
    
    if len(g) == 9504:
        continue
    print n, len(g)    
    for nn, gg in g.groupby('TimePiece'):
        if len(gg) == 66:
            continue
        print nn, len(gg)

In [56]:
66 * 144

9504

In [19]:
temp_df = temp_df.merge(all_traffic, on=['id', 'Date', 'TimePiece'], how='left')
temp_df.describe()

,id,TimePiece,Weather,traffic
count,247104.00000,247104.000000,247104.000000,201934.000000
mean,33.50000,72.500000,2.379274,0.094944
std,19.05041,41.568301,1.928301,0.043510
min,1.00000,1.000000,0.000000,0.000000
25%,17.00000,36.750000,1.000000,0.066667
50%,33.50000,72.500000,2.000000,0.093333
75%,50.00000,108.250000,4.000000,0.122040
max,66.00000,144.000000,9.000000,1.000000


In [28]:
all_sd_df.supply.dtype

dtype('int32')

In [20]:
final_df = temp_df.merge(all_sd_df, on=['id', 'Date', 'TimePiece'], how='left')

In [21]:
final_df['week'] = pd.Series([datetime.strptime(e, "%Y-%m-%d").weekday() for e in final_df.Date])
final_df['work'] = pd.Series([e < 5 for e in final_df.week])


In [22]:
final_df.describe()

,id,TimePiece,Weather,traffic,demand,supply,gap,week,work
count,247104.00000,247104.000000,247104.000000,201934.000000,171415.000000,171415.000000,171415.000000,247104.000000,247104
mean,33.50000,72.500000,2.379274,0.094944,53.079363,43.596908,9.482455,3.153846,0.692308
std,19.05041,41.568301,1.928301,0.043510,113.876910,82.040953,50.229963,1.974696,0.461539
min,1.00000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,False
25%,17.00000,36.750000,1.000000,0.066667,4.000000,3.000000,0.000000,1.000000,0
50%,33.50000,72.500000,2.000000,0.093333,12.000000,10.000000,1.000000,3.000000,1
75%,50.00000,108.250000,4.000000,0.122040,48.000000,42.000000,4.000000,5.000000,1
max,66.00000,144.000000,9.000000,1.000000,4362.000000,1084.000000,3872.000000,6.000000,True


In [32]:
for c in final_df.columns:
    print c, final_df[c].dtype
    

id int16
Date object
TimePiece int16
week int64
work bool
Weather int8
demand float64
supply float64
gap float64
traffic float64


In [25]:
final_df = final_df.loc[:, ['id', 'Date', 'TimePiece', 'week', 'work', 'Weather', 'demand', 'supply', 'gap', 'traffic']]

In [26]:
final_df.to_csv(full_file, index=False)